689. Maximum Sum of 3 Non-Overlapping Subarrays
     Hard
     Topics
     Companies
     Given an integer array nums and an integer k, find three non-overlapping subarrays of length k with maximum sum and return them.

Return the result as a list of indices representing the starting position of each interval (0-indexed). If there are multiple answers, return the lexicographically smallest one.

Example 1:

Input: nums = [1,2,1,2,6,7,5,1], k = 2
Output: [0,3,5]
Explanation: Subarrays [1, 2], [2, 6], [7, 5] correspond to the starting indices [0, 3, 5].
We could have also taken [2, 1], but an answer of [1, 3, 5] would be lexicographically larger.
Example 2:

Input: nums = [1,2,1,2,1,2,1,2,1], k = 2
Output: [0,2,4]

Constraints:

1 <= nums.length <= 2 \* 104
1 <= nums[i] < 216
1 <= k <= floor(nums.length / 3)

---

give best time and space complexity code in rust,give variables usefull name, add comments to explain the code way more clear,also add code to run example case, use below typing

---

impl Solution {
    pub fn max_sum_of_three_subarrays(nums: Vec<i32>, k: i32) -> Vec<i32> {
        
    }
}

In [ ]:
impl Solution {
    pub fn max_sum_of_three_subarrays(nums: Vec<i32>, k: i32) -> Vec<i32> {
        let k = k as usize;
        let windows: Vec<i32> = nums
          .windows(k)
          .enumerate()
          .scan((0, 0), |f, (i, w)| {
            if i == 0 {
                *f = (w.iter().sum::<i32>(), w[0]);
            } else {
                let drop = f.1;
                let add = w[w.len() - 1];
                *f = (f.0 - drop + add, w[0]);
            }
            Some(f.0)
          })
          .collect();

        let dp: Vec<(i32, usize, usize)> = windows
          .iter()
          .enumerate()
          .scan((i32::MIN, 0_usize, 0_usize), |max, (i, &w)| {
            if w > max.0 {
                *max = (w, 0_usize, i);
            }
            Some(*max)
          })
          .collect();

        let dp: Vec<Vec<(i32, usize, usize)>> = std::iter::successors(Some((dp, 0_usize)), |(prev_mx, i)| {
            let i = *i + 1;
            let start = i * k;
            let max_it = windows
                .iter()
                .enumerate()
                .skip(start)
                .map(|(i, &w)| (prev_mx[i - k].0 + w, prev_mx[i - k].2, i))
                .scan((i32::MIN, 0_usize, 0_usize), |max, (cur, prev_i, i)| {
                  if cur > max.0 {
                    *max = (cur, prev_i, i);
                  }
                  Some(*max)
                });
            let cur_max = std::iter::repeat((i32::MIN, 0, 0))
                .take(start)
                .chain(max_it)
                .collect();
            Some((cur_max, i))
        })
        .take(3)
        .map(|(arr, _)| arr)
        .collect();
        
        let mut res: Vec<i32> = dp
          .into_iter()
          .rev()
          .scan(-1, |prev_idx, arr| {
            if *prev_idx < 0 {  // first
                let (_, prev_i, i) = arr
                  .into_iter()
                  .rev()
                  .max_by_key(|&(n, prev_i, i)| n)
                  .unwrap();
                *prev_idx = prev_i as i32;
                Some(i as i32)
            } else {
                let (_, prev_i, i) = arr[*prev_idx as usize];
                *prev_idx = prev_i as i32;
                Some(i as i32)
            }
          })
          .collect();
        res.reverse();
        res
    }
}

In [ ]:
impl Solution {
    pub fn max_sum_of_three_subarrays(nums: Vec<i32>, k: i32) -> Vec<i32> {
        let k = k as usize;
        let mut sums = Vec::with_capacity(nums.len() - k + 1);
        let mut curr = nums.iter().copied().take(k).sum::<i32>();
        sums.push(curr);
        for i in k..nums.len() {
            curr = curr + nums[i] - nums[i - k];
            sums.push(curr);
        }
        let mut sums2 = Vec::with_capacity(sums.len());
        let mut prev_max_sum = (i32::MIN, 0);
        for i in k..sums.len()-k {
            if sums[i-k] > prev_max_sum.0 {
                prev_max_sum = (sums[i-k], i-k);
            }
            sums2.push((sums[i] + prev_max_sum.0, prev_max_sum.1));
        }
        //println!("{:?} {:?}", sums, sums2);
        let mut max_sum2 = (i32::MIN, 0, 0);
        let mut max_sum3 = (i32::MIN, 0, 0, 0);
        for i in 2*k..sums.len() {
            if sums2[i-2*k].0 > max_sum2.0 {
                max_sum2 = (sums2[i-2*k].0, sums2[i-2*k].1, i-k);
            }
            if max_sum2.0 + sums[i] > max_sum3.0 {
                max_sum3 = (max_sum2.0 + sums[i], max_sum2.1, max_sum2.2, i);
            }
        }
        vec![max_sum3.1 as i32, max_sum3.2 as i32, max_sum3.3 as i32]
    }
}


In [ ]:
impl Solution {
    pub fn max_sum_of_three_subarrays(nums: Vec<i32>, k: i32) -> Vec<i32> {
        let k = k as usize;
        let mut pref_sum = vec![0; nums.len() + 1];

        for i in 0..k {
            pref_sum[k] += nums[i];
        }

        for i in (k + 1)..=nums.len() {
            pref_sum[i] = pref_sum[i - 1] + nums[i - 1] - nums[i - 1 - k];
        }

        let mut pref_sum_pref_max = vec![0; nums.len() + 1];
        pref_sum_pref_max[2 * k] = k;

        for i in (2 * k + 1)..=nums.len() {
            pref_sum_pref_max[i] = if pref_sum[i - k] > pref_sum[pref_sum_pref_max[i - 1]] {
                i - k
            } else {
                pref_sum_pref_max[i - 1]
            };
        }

        let mut pref_sum_pref_max_pref_max = vec![0; nums.len() + 1];
        pref_sum_pref_max_pref_max[3 * k] = 2 * k;

        for i in (3 * k + 1)..=nums.len() {
            pref_sum_pref_max_pref_max[i] = if
                pref_sum[i - k] + pref_sum[pref_sum_pref_max[i - k]] >
                pref_sum[pref_sum_pref_max_pref_max[i - 1]] + pref_sum[pref_sum_pref_max[pref_sum_pref_max_pref_max[i - 1]]]
            {
                i - k
            } else {
                pref_sum_pref_max_pref_max[i - 1]
            }
        }

        //return pref_sum_pref_max_pref_max.into_iter().map(|i| i as i32).collect();

        let mut max = 0;
        let mut ans = (0, 0, 0);

        for i in (3 * k)..=nums.len() {
            let a = i;
            let b = pref_sum_pref_max_pref_max[i];
            let c = pref_sum_pref_max[pref_sum_pref_max_pref_max[i]];
            let s = pref_sum[a] + pref_sum[b] + pref_sum[c];
            let new_ans = (c - k, b - k, a - k);

            if s > max {
                max = s;
                ans = new_ans;
            }

            if s == max {
                if ans > new_ans {
                    ans = new_ans;
                }
            }
        }

        vec![ans.0 as i32, ans.1 as i32, ans.2 as i32]
    }
}